### EX1: Read CSV

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(0)
%matplotlib inline

def load_data_from_file(fileName = "advertising.csv"):
    data = np.genfromtxt(fileName, dtype=None, delimiter=',', skip_header=1)
    features_X = data[:, :3]
    sales_Y = data[:, 3]
    features_X = np.c_[np.ones((len(features_X),1)), features_X]
    return features_X, sales_Y

In [14]:
# Q2
features_X, _ = load_data_from_file()
print(features_X[:5,:])

[[  1.  230.1  37.8  69.2]
 [  1.   44.5  39.3  45.1]
 [  1.   17.2  45.9  69.3]
 [  1.  151.5  41.3  58.5]
 [  1.  180.8  10.8  58.4]]


In [15]:
# Q3
_, sales_Y = load_data_from_file()
print(sales_Y.shape)

(200,)


### EX2: Create individual

In [16]:
def create_individual(n=4, bound=10):
    individual = []
    for _ in range(n):
        individual.append(random.uniform(-bound/2, bound/2))
    return individual

individual = create_individual()
print(individual)

[3.4442185152504816, 2.5795440294030243, -0.79428419169155, -2.4108324970703663]


### EX3: Fitness function

In [17]:
features_X, sales_Y = load_data_from_file()

def compute_loss(individual):
    theta = np.array(individual)
    y_hat = features_X.dot(theta)
    loss = np.multiply((y_hat-sales_Y), (y_hat-sales_Y)).mean()
    return loss

def compute_fitness(individual):
    loss = compute_loss(individual)
    fitness_value = 0
    # ************ your code here ************
    fitness_value = 1/loss
    return fitness_value

In [18]:
# Q4
features_X, sales_Y = load_data_from_file()
individual = [4.09, 4.82, 3.10, 4.02]
fitness_score = compute_fitness(individual)
print(fitness_score)

1.0186001912541925e-06


### EX4: Crossover

In [19]:
def crossover(individual1, individual2, crossover_rate = 0.9):
    individual1_new = individual1.copy()
    individual2_new = individual2.copy()
    # *************** Your code here **************
    n = len(individual1)
    for i in range(0, n):
        if random.random() < crossover_rate:
            individual1_new[i] = individual2[i]
            individual2_new[i] = individual1[i]
    return individual1_new, individual2_new

In [20]:
# Q5
individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57,-0.79,-2.41]
individual1, individual2 = crossover(individual1, individual2, 2.0)
print("individual1: ", individual1)
print("individual2: ", individual2)

individual1:  [3.44, 2.57, -0.79, -2.41]
individual2:  [4.09, 4.82, 3.1, 4.02]


### EX5: Mutation

In [21]:
def mutate(individual, mutation_rate = 0.05):
    individual_m = individual.copy()
    # ******** Your code here ******************
    for i in range(0, len(individual)):
        if random.random() > mutation_rate:
            individual_m[i] = random.randint(0,1)
    return individual_m

In [22]:
# Q6
before_individual = [4.09, 4.82, 3.10, 4.02]
after_individual = mutate(individual, mutation_rate = 2.0)
print(before_individual == after_individual)

True


### EX6: Initialize Population

In [32]:
def initialize_population(m):
    population = [create_individual() for _ in range(m)]
    return population

### EX7: Selection

In [23]:
def selection(sorted_old_population, m = 100):
    index1 = random.randint(0, m-1)
    while True:
        index2 = random.randint(0, m-1)
        if (index2 != index1):
            break
    individual_s = sorted_old_population[index1]
    if index2 > index1:
        individual_s = sorted_old_population[index2]
    return individual_s

[5, 6]

### EX8: Create new population

In [36]:
def create_new_population(old_population, elitism=2, gen=1):
    m = len(old_population)
    sorted_population = sorted(old_population, key=compute_fitness)

    if gen%1 == 0:
        print("Best loss:", compute_loss(sorted_population[m-1]), "with chromsome: ", sorted_population[m-1])
    
    new_population = []
    while len(new_population) < m-elitism:
        # selection
        individual_s1 = selection(sorted_population, m)
        individual_s2 = selection(sorted_population, m)

        # crossover
        individual_c1, individual_c2 = crossover(individual_s1, individual_s2)
        
        # mutation
        individual_m1 = mutate(individual_c1)
        individual_m2 = mutate(individual_c2)
        
        new_population.append(individual_m1)
        new_population.append(individual_m2)

    # copy elitism chromosomes that have best fitness score to the next generation
    for ind in sorted_population[m-elitism:]:
        new_population.append(ind)
    
    return new_population, compute_loss(sorted_population[m-1])

In [37]:
# Q7
individual1 = [4.09, 4.82, 3.10, 4.02]
individual2 = [3.44, 2.57,-0.79,-2.41]
old_population = [individual1, individual2]
new_population, _ = create_new_population(old_population, elitism=2, gen=1)

Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]


### EX9: All in one - GA

In [39]:
def run_GA():
    n_generations = 100
    m = 600
    features_X, sales_Y = load_data_from_file()
    population = initialize_population(m)
    losses_list = []
    for i in range(n_generations):
        new_population, loss = create_new_population(old_population, elitism=2, gen=1)
        losses_list.append(loss)
    return losses_list

losses_list = run_GA()

Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79, -2.41]
Best loss: 123415.051528805 with chromsome:  [3.44, 2.57, -0.79,

In [ ]:
import matplotlib.pyplot as plt

def visualize_loss(losses_list):
    

losses_list = run_GA()
visualize_loss(losses_list)